In [1]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x * x * y + y + 2

sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [2]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [3]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

In [4]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [5]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [6]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


In [7]:
tf.reset_default_graph()

# Exercise 12

In [2]:
def logistic_regression(w, b, x):
    weighted_input_plus_bias = w * x + b
    sigmoid = 1 / (1 + tf.exp(-weighted_input_plus_bias))
    return sigmoid

In [3]:
from sklearn.datasets import make_moons

moon_X, moon_y = make_moons()

In [4]:
moon_X.shape

(100, 2)

In [5]:
moon_y

array([1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0])

In [6]:
moon_y.shape

(100,)

In [8]:
number_of_features = moon_X.shape[1]
w = tf.Variable(tf.random_normal([number_of_features], name="weights"))
b = tf.Variable(0.0, name="bias")
x = tf.placeholder(tf.float32, shape=(None, number_of_features), name="input")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
output = logistic_regression(w, b, x)

error = output - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
training_op = optimizer.minimize(mse)

In [19]:
import numpy as np

epochs = 500
batch_size = 10
n_batches = int(np.ceil(moon_X.shape[0] / batch_size))

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch_index in range(n_batches):
            batch_X = moon_X[batch_index * batch_size:(batch_index + 1) * batch_size]
            #print("batch_X shape:", batch_X.shape)
            batch_y = moon_y[batch_index * batch_size:(batch_index + 1) * batch_size].reshape((-1, 1))
            #print("batch_Y shape:", batch_y.shape)
            t, m = sess.run([training_op, mse], feed_dict={x: batch_X, y: batch_y})
            if batch_index == 0: print("Mean squared error:", m)

Mean squared error: 0.261194
Mean squared error: 0.253638
Mean squared error: 0.246233
Mean squared error: 0.239077
Mean squared error: 0.232266
Mean squared error: 0.225878
Mean squared error: 0.219969
Mean squared error: 0.214565
Mean squared error: 0.209665
Mean squared error: 0.205249
Mean squared error: 0.201285
Mean squared error: 0.197731
Mean squared error: 0.194546
Mean squared error: 0.191689
Mean squared error: 0.189119
Mean squared error: 0.186804
Mean squared error: 0.18471
Mean squared error: 0.182812
Mean squared error: 0.181084
Mean squared error: 0.179505
Mean squared error: 0.178058
Mean squared error: 0.176726
Mean squared error: 0.175496
Mean squared error: 0.174356
Mean squared error: 0.173295
Mean squared error: 0.172305
Mean squared error: 0.171377
Mean squared error: 0.170506
Mean squared error: 0.169685
Mean squared error: 0.168909
Mean squared error: 0.168174
Mean squared error: 0.167476
Mean squared error: 0.166811
Mean squared error: 0.166177
Mean squared er

Mean squared error: 0.14291
Mean squared error: 0.142908
Mean squared error: 0.142907
Mean squared error: 0.142905
Mean squared error: 0.142903
Mean squared error: 0.142902
Mean squared error: 0.1429
Mean squared error: 0.142899
Mean squared error: 0.142897
Mean squared error: 0.142896
Mean squared error: 0.142894
Mean squared error: 0.142893
Mean squared error: 0.142891
Mean squared error: 0.14289
Mean squared error: 0.142889
Mean squared error: 0.142888
Mean squared error: 0.142886
Mean squared error: 0.142885
Mean squared error: 0.142884
Mean squared error: 0.142883
Mean squared error: 0.142882
Mean squared error: 0.142881
Mean squared error: 0.14288
Mean squared error: 0.142879
Mean squared error: 0.142878
Mean squared error: 0.142877
Mean squared error: 0.142876
Mean squared error: 0.142875
Mean squared error: 0.142875
Mean squared error: 0.142874
Mean squared error: 0.142873
Mean squared error: 0.142872
Mean squared error: 0.142872
Mean squared error: 0.142871
Mean squared error: